In [ ]:
#挂载云盘 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#进入项目目录
print('original document')
!ls
import os
os.chdir('/content/drive/My Drive/colab/2020年C题')
print('current document')
!ls

original document
 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1.ipynb
 data.rar
 get_plot_data_fastica.xlsx
 get_plot_data.xlsx
 unrar.ipynb
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx
current document
 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1.ipynb
 data.rar
 get_plot_data_fastica.xlsx
 get_plot_data.xlsx
 unrar.ipynb
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx


In [ ]:
#设置参数
import numpy as np
import pandas as pd
s=1#第几个被试
train_data_path='附件1-P300脑机接口数据/S%d/S%d_train_data.xlsx'%(s,s)
train_event_path='附件1-P300脑机接口数据/S%d/S%d_train_event.xlsx'%(s,s)
cut_begin,cut_end=200,500#ms

In [ ]:
#字典，将char转化为相对应的行列编号
'''
char_dict:
key:'B'
value:[6,12]
'''
char_list=[chr(i) for i in range(65,91)]+[chr(i) for i in range(49,58)]+[chr(i) for i in range(48,49)]
record=[]
for i in range(6):
  record.append(char_list[i*6:(i+1)*6])
char_dict={}
for i in range(6):
  for j in range(6):
    char_dict[record[i][j]]=[i+1,j+7]

#字典，将表头转化为相对应char
'''
event_dict:
key:102
value:'B'
'''
event_dict={}
record=[]
for i in range(len(char_list)):
  event_dict[i+101]=char_list[i]

In [ ]:
from scipy import stats #scipy中的stats可以做统计推断
#实现对原始数据对预处理 去均值等
def raw_data_preprocessing(data):
  #data=(data-data.min())/(data.max()-data.min())
  data=data-data.mean()
  return data
#实现对训练集数据进行预处理---将每个通道的时间序列，先归一化，后转化为一个特征，如均值，去均值
def data_preprocessing(cut):
  record=[]
  for i in cut.columns:
    temp_column=cut[i]
    temp_column=(temp_column-temp_column.min())/(temp_column.max()-temp_column.min())#预处理--归一化
    mu = np.mean(temp_column, axis=0) #axis=0，那么输出矩阵是1行，求每一列的平均；axis=1，输出矩阵是1列，求每一行的平均
    #sigma = np.std(temp_column, axis=0) #求标准差，这里除的是N
    #skew = stats.skew(temp_column) #求偏度
    #kurtosis = stats.kurtosis(temp_column) #求峰度 
    #median=temp_column.median()
    #quantile=temp_column.quantile(q=0.5)
    #skew=temp_column.skew()
    #mean_column=temp_column.mean()
    record.append(mu)
  return np.array(record)

In [ ]:
from sklearn.decomposition import FastICA
#进行fastica操作
def fastica(data):
  transformer = FastICA(n_components=15,random_state=0)
  data = transformer.fit_transform(data)
  data=pd.DataFrame(data)
  return data
#是否是当前的char
def is_current_char(action,char_dict,current_char):
  result=0
  if action in char_dict[current_char]:
    result=1
  return result
#根据当前char生成对应数据集
def data_generate(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]
  data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    record.append(cut_preprocessing)
    record2.append(label)
  record=np.array(record)
  record2=np.array(record2)
  #print(event.shape,record.shape,record2.shape,'如果前三者行数相等，说明数据集构建没有问题')
  return record,record2
#获取一组可以画图的数据 原始数据
def get_plot_data(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  保存为excel
  '''
  record=[]
  record2=[]
  data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  if current_char!='B':
    return
  writer=pd.ExcelWriter('get_plot_data_fastica.xlsx')
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    #cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    cut.to_excel(writer,sheet_name='id=%d,label=%d'%(i,label),index=0)
    writer.save() 
    writer.close()

In [ ]:
#对所有sheet都生成对应的训练测试集，并汇总
#代码结构还是有点差，比赛期间没时间改啦
def data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end):
  df = pd.read_excel(train_data_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  train_label_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    data=raw_data_preprocessing(data)
    #print(data.head())
    event=pd.read_excel(train_event_path,sheet_name) 
    #print(event.head())
    train_data,train_label=data_generate(data,event,cut_begin,cut_end)
    #get_plot_data(data,event,cut_begin,cut_end)#获取画图数据
    train_data_record.append(train_data)
    train_label_record.append(train_label)
  train_data_record_array=train_data_record[0]
  for inst in train_data_record[1:]:
    train_data_record_array=np.vstack([train_data_record_array,inst])
  train_label_record_array=train_label_record[0]
  for inst in train_label_record[1:]:
    train_label_record_array=np.hstack([train_label_record_array,inst])
  return train_data_record_array,train_label_record_array

In [ ]:
real_train_data,real_train_label=data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_fastica.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_fastica.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_fastica.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_fastica.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [ ]:
print('通过shape看一下数据集的特征和标签有没有什么问题',real_train_data.shape,real_train_label.shape)

In [ ]:
#dataset=pd.DataFrame(real_train_data)
#plt.figure(figsize=(10,100))
#sns.heatmap(dataset.iloc[:,:],annot=False,cmap="YlGnBu")
#plt.show()

In [ ]:
#归一化 
real_train_data_norm=(real_train_data-real_train_data.min())/(real_train_data.max()-real_train_data.min())
#不归一化 
#real_train_data_norm=real_train_data
real_train_data_norm[1:5]

In [ ]:
## 下采样
train_y=np.array(real_train_label)
train=pd.concat([pd.DataFrame(real_train_data_norm),pd.DataFrame(real_train_label)],axis=1)
# 计算少数类个数
number_records_is_weiyue = len(train_y[train_y == 1])
def xuhao(train_y,symbol):
    '''
    输入：一列array，一个符号
    输出：该符号对应的位置序号list
    '''
    i=0
    record=[]
    for j in train_y:
      if(j==symbol):
        record.append(i)
      i=i+1
    return record
# 取得少数类样本的索引
is_weiyue_indices = np.array(xuhao(train_y,1))
# 取得多数类样本的索引
normal_indices = np.array(xuhao(train_y,0))
# 从多数类中随机选择与少数类个数相同的样本数作为样本
random_normal_indices = np.random.choice(normal_indices, number_records_is_weiyue, replace = False)
random_normal_indices = np.array(random_normal_indices)
# 合并随机取得的0类和全部的1类的索引
under_sample_indices = np.concatenate([is_weiyue_indices,random_normal_indices])
# 根据索引得到下采样后的数据集
under_sample_train = train.iloc[under_sample_indices,:]
#打乱
from sklearn.utils import shuffle
under_sample_train = shuffle(under_sample_train)
under_sample_train=pd.DataFrame(under_sample_train.values)
under_sample_train#最终的数据集

In [ ]:
from sklearn.model_selection import cross_val_score
X,Y=under_sample_train.iloc[:,:-1],under_sample_train.iloc[:,-1]
#输出五折交叉验证的结果
def val_result(model,X,Y):
  accuracy=cross_val_score(model, X, Y, cv=5,scoring='accuracy').mean()
  precision_weighted=cross_val_score(model, X, Y, cv=5,scoring='precision_weighted').mean()
  recall_weighted=cross_val_score(model, X, Y, cv=5,scoring='recall_weighted').mean()
  f1_weighted=cross_val_score(model, X, Y, cv=5,scoring='f1_weighted').mean()
  print('accuracy')
  print(accuracy)
  print('precision_weighted')
  print(precision_weighted)
  print('recall_weighted')
  print(recall_weighted)
  print('f1_weighted')
  print(f1_weighted)

In [ ]:
#逻辑回归
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
val_result(model,X,Y)

In [ ]:
#决策树方法
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
model=DecisionTreeClassifier()
val_result(model,X,Y)

In [ ]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
val_result(model,X,Y)

In [ ]:
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
val_result(model,X,Y)

In [ ]:
#knn k近邻分类算法
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
val_result(model,X,Y)

In [ ]:
#神经网络分类方法
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8, ),random_state=1, max_iter=300)
val_result(model,X,Y)

In [ ]:
# 支持向量机SVM
from sklearn.svm import SVC  
model = SVC(kernel='rbf') 
val_result(model,X,Y)

In [ ]:
#线性支持向量机SVM
from sklearn.svm import LinearSVC
model = LinearSVC() 
val_result(model,X,Y)

In [ ]:
#使用bagging的集成策略分类 元分类器是KNN
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=7),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)

In [ ]:
#使用bagging的集成策略分类 元分类器是logistic
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=LogisticRegression(),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)


In [ ]:
#划分训练集和验证集
cut=len(under_sample_train)//10
x_train,y_train,x_val,y_val=under_sample_train.iloc[:-cut,:-1],under_sample_train.iloc[:-cut,-1],under_sample_train.iloc[-cut:,:-1],under_sample_train.iloc[-cut:,-1]
x_train,y_train,x_val,y_val=x_train.values,y_train.values,x_val.values,y_val.values
print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

In [ ]:
def result(train_data,train_labels,test_data,test_labels,model):
    model.fit(train_data,train_labels)
    #predict_target=model.predict(train_data)
    #在训练集上的预测效果,
    #print("训练集:")
    #print("预测正确数量,训练集样本量:")
    #print(sum(predict_target == train_labels),len(train_labels))
    #print("精确度等指标：")
    #print(metrics.classification_report(train_labels,predict_target))
    #print("混淆矩阵：")
    #print(metrics.confusion_matrix(train_labels,predict_target))
    #dataset=pd.DataFrame(metrics.confusion_matrix(train_labels,predict_target))
    #plt.figure(figsize=(4,3))
    #sns.heatmap(dataset.iloc[:,:],annot=True,cmap="YlGnBu")
    #plt.show()

    #print("在训练集上的预测效果:")
    #plt.plot(train_labels[50:80]) 
    #plt.plot(predict_target[50:80],'o')
    #plt.legend(('y_train','y_predict'))
    #plt.title('train')
    #plt.show()

    #在验证集上的预测效果

    print("验证集:")
    predict_target2=model.predict(test_data)
    print("预测正确数量,验证集样本量:")
    print(sum(predict_target2 == test_labels),len(test_labels))
    print("精确度等指标：")
    print(metrics.classification_report(test_labels,predict_target2))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels,predict_target2))
    dataset=pd.DataFrame(metrics.confusion_matrix(test_labels,predict_target2))
    plt.figure(figsize=(4,3))
    sns.heatmap(dataset.iloc[:,:],annot=True,cmap="YlGnBu")
    plt.show()

    print("在验证集上的预测效果:")
    plt.plot(test_labels) 
    plt.plot(predict_target2,'o')
    plt.legend(('y_test','y_predict'))
    plt.title('test')
    plt.show()

In [ ]:
#随机森林方法
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#knn k近邻分类算法
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#神经网络分类方法
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8, ),random_state=1, max_iter=300)
result(x_train,y_train,x_val,y_val,model)

In [ ]:
# 支持向量机SVM
from sklearn.svm import SVC  
model = SVC(kernel='rbf') 
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#线性支持向量机SVM
from sklearn.svm import LinearSVC
model = LinearSVC() 
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#决策树方法
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
model=DecisionTreeClassifier()
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#使用bagging的集成策略分类 元分类器是KNN
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=7),
                        n_estimators=10, random_state=0)
result(x_train,y_train,x_val,y_val,model)

In [ ]:
#logistic回归
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0,l1_ratio=1)
result(x_train,y_train,x_val,y_val,model)